In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w6-p2/X_test.csv
/kaggle/input/2021-ai-w6-p2/train.csv
/kaggle/input/2021-ai-w6-p2/sample_submit.csv


In [2]:
train=pd.read_csv("/kaggle/input/2021-ai-w6-p2/train.csv")
test=pd.read_csv("/kaggle/input/2021-ai-w6-p2/X_test.csv")
submit=pd.read_csv("/kaggle/input/2021-ai-w6-p2/sample_submit.csv")

In [3]:
train

,Day,Hour,Minute,DHI,DNI,WS,RH,T,TARGET
0,0,0,0,0,0,1.5,69.08,-12,0.000000
1,0,0,30,0,0,1.5,69.06,-12,0.000000
2,0,1,0,0,0,1.6,71.78,-12,0.000000
3,0,1,30,0,0,1.6,71.75,-12,0.000000
4,0,2,0,0,0,1.6,75.20,-12,0.000000
...,...,...,...,...,...,...,...,...,...
9994,208,5,0,0,0,0.5,64.12,12,0.000000
9995,208,5,30,26,383,0.8,56.18,13,5.630068
9996,208,6,0,41,578,1.1,47.46,15,13.887196
9997,208,6,30,52,699,1.4,44.51,17,23.269925


In [4]:
x=train.drop(["TARGET","Day","Minute"],axis=1)
test=test.drop(["Day","Minute","ID"],axis=1)
y=train["TARGET"]

In [5]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
scaler.fit(x)
x=scaler.transform(x)
test=scaler.transform(test)

In [6]:
import torch
seed=777

torch.manual_seed(seed)

device='cuda' if torch.cuda.is_available() else 'cpu'

if device=='cuda':
  torch.cuda.manual_seed_all(seed)

In [7]:
x=torch.FloatTensor(np.array(x))
y=torch.FloatTensor(np.array(y))
test=torch.FloatTensor(np.array(test))

In [8]:
linear1=torch.nn.Linear(6,128)
linear2=torch.nn.Linear(128,256)
linear3=torch.nn.Linear(256,128)
linear4=torch.nn.Linear(128,1)

relu=torch.nn.ReLU()

dropout=torch.nn.Dropout(p=0.1)

In [9]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)

Parameter containing:
tensor([[ 0.0434,  0.0947, -0.0569,  0.0032, -0.0840,  0.0953, -0.0479, -0.1396,
          0.0098, -0.0924, -0.0797,  0.0531, -0.1087,  0.0687,  0.2002, -0.0341,
          0.0805, -0.0782, -0.0261, -0.1066, -0.1150,  0.0507,  0.0735,  0.0536,
         -0.1103, -0.1231,  0.1745, -0.0417,  0.1407, -0.1268, -0.0963,  0.0303,
         -0.1270, -0.0865, -0.1886, -0.1912,  0.1035, -0.1495, -0.1231,  0.2101,
          0.1191,  0.0395, -0.2132,  0.1743, -0.0467, -0.0660,  0.1550,  0.0701,
         -0.1223,  0.0826, -0.2138, -0.1809, -0.1040,  0.0243, -0.1861, -0.0538,
          0.1841, -0.1273,  0.1478, -0.1235, -0.0475, -0.1719,  0.1976, -0.1000,
         -0.1980, -0.0626,  0.0335,  0.0212,  0.1089,  0.0428,  0.0595, -0.0980,
          0.1670, -0.1963,  0.1273,  0.1525,  0.2070, -0.1513, -0.0638, -0.1432,
         -0.1969,  0.0669,  0.2047, -0.1073, -0.1918, -0.2066, -0.1359, -0.0852,
         -0.0656,  0.0079, -0.1068, -0.1299,  0.1127,  0.0555,  0.0014, -0.1480,
      

In [10]:
model=torch.nn.Sequential(linear1,relu,dropout,linear2,relu,dropout,linear3,relu,dropout,linear4).to(device)
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)
loss=torch.nn.MSELoss().to(device)

In [11]:
model.train()
for epoch in range(1000):
  optimizer.zero_grad()
  h=model(x.to(device))
  cost=loss(h,y.unsqueeze(1).to(device))
  cost.backward()
  optimizer.step()
  if epoch%100==0:
    print(epoch,cost.item())

0 1016.7341918945312
100 36.381752014160156
200 18.1806640625
300 12.773265838623047
400 9.5596284866333
500 8.536458015441895
600 7.804405212402344
700 7.123076915740967
800 6.460417747497559
900 6.5486226081848145


In [12]:
pred=list()

with torch.no_grad():
  model.eval()
  y_test=model(test.to(device))
  a,b=torch.max(y_test,dim=1)
  pred.extend(a.tolist())

In [13]:
submit

,ID,TARGET
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0
...,...,...
2476,2476,0
2477,2477,0
2478,2478,0
2479,2479,0


In [14]:
submit["TARGET"]=pred
submit.to_csv("submit.csv",index=False)